In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import selenium.webdriver.support.ui as ui
import time
chrome_options = Options()  
chrome_options.add_argument("--headless")
br = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'./chromedriver.exe')

In [2]:
# Warning: code takes more than an hour to run since we have to wait until every
# page's Javascript content loads. That takes around 1-2 seconds normally, but
# sometimes may take one second longer than that, causing the program to crash.
# We use 5 seconds just in case, to avoid any possible hangups in the process.

# Scrapes median income
def scraping_states(state, row):
  
    br.get("https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml")
    wait = ui.WebDriverWait(br,15)
    time.sleep(2)
    # Types search terms for a city
    element = wait.until(lambda br:br.find_element_by_id('cfsearchtextbox'))
    element.clear()
    element.send_keys(state)

    # Submits form
    element =  wait.until(lambda br: br.find_element_by_xpath('//*[@id="communityfactssubmit"]'))
    element.click()

    # Clicking the income tab
    time.sleep(2.5)
    element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="leftnav"]/a[7]'))
    element.click()

    # Getting table link
    time.sleep(2.5)
    element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="cf-content"]/div[2]/div[1]/ul[1]/li[2]/div/a'))
    href = element.get_attribute('href')
    br.get(href)

    #
    for i in range(2, 9):
        time.sleep(5)
            
        # Append unemployment rate
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="data"]/tbody/tr[13]/td[1]'))
        incomeDF.iloc[row, i-2] = element.text
        
        
        # Stop from getting nonexistent element
        if i == 8:
            break
        
        # Get previous year
        element = wait.until(lambda br:br.find_element_by_xpath('//*[@id="year_selector_content"]/ul/li['+ str(i) +']/a'))
        element.click()


In [3]:
states = [line.rstrip('\n\r') for line in open('./states.txt')]

# Setting up the column names for each data frame
incomeDF = pd.DataFrame(columns=["State", "Median Income 2016", "Median Income 2015", "Median Income 2014", "Median Income 2013",
                  "Median Income 2012", "Median Income 2011", "Median Income 2010"])


# Set the amount of rows per data frame
data = pd.DataFrame({"State": range(len(states))})
incomeDF = incomeDF.append(data)


# Input the name of a city per row in each data frame
for i, row in incomeDF.iterrows():
    row.loc['State'] = states[i]
    

# Reverse the column order (ie 2010 to 2016)
incomeDF = incomeDF.iloc[:, ::-1]


for i, state in enumerate(states):
    scraping_states(state, i)
    print(state + " " + str(i)) # Done to see what city we're currently scraping

# Setting the "City" as the index column for all dataframes
incomeDF = incomeDF.set_index('State')

# Close the webdriver instance
br.quit()

Alabama 0
Alaska 1
Arizona 2
Arkansas 3
California 4
Colorado 5
Connecticut 6
Delaware 7
Florida 8
Georgia 9
Hawaii 10
Idaho 11
Illinois 12
Indiana 13
Iowa 14
Kansas 15
Kentucky 16
Louisiana 17
Maine 18
Maryland 19
Massachusetts 20
Michigan 21
Minnesota 22
Mississippi 23
Missouri 24
Montana 25
Nebraska 26
Nevada 27
New Hampshire 28
New Jersey 29
New Mexico 30
New York 31
North Carolina 32
North Dakota 33
Ohio 34
Oklahoma 35
Oregon 36
Pennsylvania 37
Rhode Island 38
South Carolina 39
South Dakota 40
Tennessee 41
Texas 42
Utah 43
Vermont 44
Virginia 45
Washington 46
West Virginia 47
Wisconsin 48
Wyoming 49


In [4]:
# Creating CSV files per dataframe
incomeDF = incomeDF.reset_index()
incomeDF.to_csv('cities_median_income.csv', encoding='utf-8', index=False)